In [1]:
# imports
import yaml

Modifiable variables

In [2]:
"""
make sure that there are csv files named 
f'{trainset_name}.csv' and f'{testset_name}.csv' in the csvs folder
"""

trainset_name = 'ETTh2'
testset_name = 'ETTh2_test'

model_name = 'moirai_1.0_R_small'
run_name = 'test-02'

Train & validation datasets vreation from .csv

In [3]:
train_val_creation_command = f"python -m uni2ts.data.builder.simple \
                                {trainset_name} ./csvs/{trainset_name}.csv \
                                    --dataset_type wide --date_offset '2017-10-23 23:00:00'"

test_creation_command = f"python -m uni2ts.data.builder.simple \
                            {testset_name} ./csvs/{testset_name}.csv \
                                --dataset_type wide"

!{train_val_creation_command}
!{test_creation_command}

Saving the dataset (1/1 shards): 100%|███| 7/7 [00:00<00:00, 1174.17 examples/s]


Load the config files

In [4]:
finetune_config_path = './conf/finetune/default.yaml'
eval_config_path = './conf/eval/default.yaml'

with open(finetune_config_path, 'r') as file:
    finetune_config = yaml.load(file, Loader=yaml.FullLoader)

with open(eval_config_path, 'r') as file:
    eval_config = yaml.load(file, Loader=yaml.FullLoader)

Create the data config files

In [5]:
# load sample data yaml files
train_sample_data_cfg_path = './conf/finetune/data/etth1.yaml'
val_sample_data_cfg_path = './conf/finetune/val_data/etth1.yaml'
test_sample_data_cfg_path = './conf/eval/data/etth1_test.yaml'

with open(train_sample_data_cfg_path, 'r') as file:
    train_sample_data_cfg = yaml.load(file, Loader=yaml.FullLoader)

with open(val_sample_data_cfg_path, 'r') as file:
    val_sample_data_cfg = yaml.load(file, Loader=yaml.FullLoader)

with open(test_sample_data_cfg_path, 'r') as file:
    test_sample_data_cfg = yaml.load(file, Loader=yaml.FullLoader)

# update the config files with the new dataset names
train_sample_data_cfg['dataset'] = trainset_name
val_sample_data_cfg['_args_']['dataset'] = trainset_name + '_eval'
test_sample_data_cfg['dataset_name'] = testset_name

# save the updated config files
train_data_cfg_path = f'./conf/finetune/data/{trainset_name.lower()}.yaml'
val_data_cfg_path = f'./conf/finetune/val_data/{trainset_name.lower()}.yaml'
test_data_cfg_path = f'./conf/eval/data/{testset_name.lower()}.yaml'

with open(train_data_cfg_path, 'w') as file:
    yaml.dump(train_sample_data_cfg, file)

with open(val_data_cfg_path, 'w') as file:
    yaml.dump(val_sample_data_cfg, file)

with open(test_data_cfg_path, 'w') as file:
    yaml.dump(test_sample_data_cfg, file)

Edit the config files

In [6]:
eval_config['defaults'][0]['model'] = model_name
eval_config['defaults'][1]['data'] = testset_name.lower()
eval_config['run_name'] = run_name

with open(eval_config_path, 'w') as file:
    yaml.dump(eval_config, file)

In [7]:
finetune_config['defaults'][0]['model'] = model_name
finetune_config['defaults'][1]['data'] = trainset_name.lower()
finetune_config['defaults'][2]['val_data'] = trainset_name.lower()
finetune_config['run_name'] = run_name

eval_config['defaults'][0]['model'] = model_name
# eval_config['defaults'][1]['data'] = testset_name.lower()
eval_config['run_name'] = run_name

# save the updated config files
with open(finetune_config_path, 'w') as file:
    yaml.dump(finetune_config, file)

with open(eval_config_path, 'w') as file:
    yaml.dump(eval_config, file)

Finetuning

In [8]:
!srun python train.py

[2024-05-19 13:34:59,073][numexpr.utils][INFO] - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2024-05-19 13:34:59,073][numexpr.utils][INFO] - NumExpr defaulting to 8 threads.
trainable parameters:
trainable params: 350,208 || all params: 14,177,736 || trainable%: 2.470126400999426
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[2024-05-19 13:35:02,113][datasets][INFO] - PyTorch version 2.3.0 available.
[2024-05-19 13:35:02,114][datasets][INFO] - JAX version 0.4.28 available.
Missing logger folder: /sise/home/liornis/thesis/research-experiments-env/finetuning/outputs/train/moirai_1.0_R_small/etth2/test-02/logs
[rank: 0] Seed set to 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type         | Params
----------------------------------------
0 | module | MoiraiModule | 14.2 M
----------------------------------------
350 K 

# Evaluation

In [9]:
!srun python eval.py

[2024-05-19 13:36:21,422][datasets][INFO] - PyTorch version 2.3.0 available.
[2024-05-19 13:36:21,424][datasets][INFO] - JAX version 0.4.28 available.
trainable parameters:
trainable params: 350,208 || all params: 14,177,736 || trainable%: 2.470126400999426
19495it [01:50, 175.82it/s]176.05it/s]
{'MSE[mean]': {None: 13.224520301345965}, 'MSE[0.5]': {None: 13.333621448283013}, 'MAE[0.5]': {None: 2.366374969547647}, 'MASE[0.5]': {None: 0.9868416217518357}, 'MAPE[0.5]': {None: 0.17066965648032656}, 'sMAPE[0.5]': {None: 0.21131828067098263}, 'MSIS': {None: 6.96813247693623}, 'RMSE[mean]': {None: 3.636553354667846}, 'NRMSE[mean]': {None: 0.20803808381485075}, 'ND[0.5]': {None: 0.1353743685955306}, 'mean_weighted_sum_quantile_loss': {None: 0.106164790897128}}
